# Digital Octopus that changes color to match background 

In this tutorial, we will be helping our digital octopus to camouflage in a forested environment. This tutorial will be run on the Jupyter Notebook environment (Callysto).

You can see that this Python solution is segmentized into many code blocks (the different grey boxes). You can run each code blocks sequentially by clicking on the **Run** button at the top of this page after you click into the code blocks. Alternatively, you can enter the **Shift-Enter** keyboard shortcut after you click into each code block to run them. 

In this tutorial you would not be doing any coding, your primary task is to run through the code and examine the output results. If you have the extra time, you can spend some effort into understanding what the code is trying to do!

In [ ]:
# import the necessary packages
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Function to display images in Jupyter/Callysto without popping up a separate window
def show_image(img, title="Image"):
    plt.figure(figsize=(10, 8))
    if len(img.shape) == 3:
        # Convert BGR (OpenCV default) to RGB (Matplotlib default)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
# select image to be background
filename = 'image1.jpg'

# read image 
img_bg = cv2.imread(filename)

# display image
show_image(img_bg, "Background image")

## Creating a mask

In [ ]:
# read octopus image (white on black)
img_oct = cv2.imread('octopus.png')

# resize to fit background image (aspect ratio not preserved)
# img_bg.shape is (height, width, channels), resize expects (width, height)
img_oct = cv2.resize(img_oct, (img_bg.shape[1], img_bg.shape[0]))

# Now create a mask of octopus and create its inverse mask also
gray = cv2.cvtColor(img_oct,cv2.COLOR_BGR2GRAY)
mask = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)[1]
mask_inv = cv2.bitwise_not(mask)

# display images side by side
plt.figure(figsize=(15, 8))

plt.subplot(1, 2, 1)
plt.imshow(mask, cmap='gray')
plt.title("Mask")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(mask_inv, cmap='gray')
plt.title("Mask inverted")
plt.axis('off')

plt.show()

## Changing colors

In [ ]:
# Store RGB values of main colors in a dataframe
# Black: [0.0.0], Red: [255,0,0], Green: [0,255,0], Blue: [0,0,255] 
main_colors = [[0,0,0], [255,0,0],[0,255,0],[0,0,255]] 
df = pd.DataFrame(main_colors, columns = ['R', 'G','B']) 

#convert BGR to RGB image
image = cv2.cvtColor(img_bg, cv2.COLOR_BGR2RGB)

h,w,bpp = image.shape

print("Processing image colors... this might take a moment.")

#Change colors of each pixel
for py in range(0,h):
    for px in range(0,w):
        R, G, B = (image[py][px][0],image[py][px][1],image[py][px][2])
        minimum = 10000
        for i in range(len(df)):
            d = abs(R- int(df.loc[i,"R"])) + abs(G- int(df.loc[i,"G"]))+ abs(B- int(df.loc[i,"B"]))
            if(d<=minimum):
                minimum = d
                result = i
        image[py][px][0]=df.loc[result,"R"]  
        image[py][px][1]=df.loc[result,"G"]  
        image[py][px][2]=df.loc[result,"B"]  
    
#convert RGB back to BGR image
img_change = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

# display image
show_image(img_change, "Changed")

## Applying mask and adding octopus to background image

In [ ]:
# cut out the area of octopus from background image
img_bg_cut = cv2.bitwise_and(img_bg,img_bg,mask = mask_inv)

# cut out the area of octopus from changed image
img_oct_cut = cv2.bitwise_and(img_change,img_change,mask = mask)

# add cut out background and octopus images together
img_add = cv2.add(img_bg_cut,img_oct_cut)

# display images
plt.figure(figsize=(20, 10))

plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(img_bg_cut, cv2.COLOR_BGR2RGB))
plt.title("Background cut")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(img_oct_cut, cv2.COLOR_BGR2RGB))
plt.title("Octopus cut")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(cv2.cvtColor(img_add, cv2.COLOR_BGR2RGB))
plt.title("Combined")
plt.axis('off')

plt.show()

In [ ]:
# save image (change to desired filename)
cv2.imwrite('image1_camo.jpg',img_add)
print("Image saved as image1_camo.jpg")